# Student Loan Risk with Deep Learning

In [4]:
pip install tensorflow


     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/376.9 MB ? eta -:--:--
   ---------------------------------------- 0.2/376.9 MB 4.6 MB/s eta 0:01:22
   ---------------------------------------- 0.7/376.9 MB 7.9 MB/s eta 0:00:48
   ---------------------------------------- 1.9/376.9 MB 13.8 MB/s eta 0:00:28
   ---------------------------------------- 3.4/376.9 MB 19.9 MB/s eta 0:00:19
    --------------------------------------- 4.9/376.9 MB 22.5 MB/s eta 0:00:17
    --------------------------------------- 5.8/376.9 MB 20.6 MB/s eta 0:00:19
    --------------------------------------- 9.0/376.9 MB 27.5 MB/s eta 0:00:14
   - -------------------------------------- 12.6/376.9 MB 46.7 MB/s eta 0:00:08
   - -------------------------------------- 16.5/376.9 MB 65.6 MB/s eta 0:00:06
   -- ------------------------------------- 20.1/376.9 MB 65.6 MB/

In [5]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [6]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [11]:
# Review the data types associated with the columns
print(loans_df.head())
print(loans_df.dtypes)


   payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0              7.4                0.70               0.00          1.9   
1              7.8                0.88               0.00          2.6   
2              7.8                0.76               0.04          2.3   
3             11.2                0.28               0.56          1.9   
4              7.4                0.70               0.00          1.9   

   alumni_success  study_major_code  time_to_completion  \
0           0.076              11.0                34.0   
1           0.098              25.0                67.0   
2           0.092              15.0                54.0   
3           0.075              17.0                60.0   
4           0.076              11.0                34.0   

   finance_workshop_score  cohort_ranking  total_loan_score  \
0                  0.9978            3.51              0.56   
1                  0.9968            3.20              0.68   
2          

In [10]:
# Check the credit_ranking value counts
print(loans_df["credit_ranking"].value_counts())


credit_ranking
1    855
0    744
Name: count, dtype: int64


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [12]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
print(y[:5])

[0 0 0 1 0]


In [13]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=["credit_ranking"])

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [15]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [17]:
# Define the the number of inputs (features) to the model
input_dim = X_train_scaled.shape[1]

# Review the number of features
print(f'Number of features: {input_dim}')

Number of features: 11


In [18]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6


# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_neurons = 1

In [19]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=input_dim))

# Add the second hidden layer
model.add(Dense(units=hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=output_neurons, activation='sigmoid'))

c:\Users\sharm\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
# Display the Sequential model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [21]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7459 - loss: 0.5465
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7523 - loss: 0.5226 
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7449 - loss: 0.5431 
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7321 - loss: 0.5545 
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7556 - loss: 0.5373
Epoch 6/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7499 - loss: 0.5312
Epoch 7/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7512 - loss: 0.5387
Epoch 8/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7541 - loss: 0.5361
Epoch 9/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7479 - loss: 0.5303 
Epoch 10/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7507 - loss: 0.5462
Epoch 11/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7564 - loss: 0.5229
Epoch 12/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7640 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print(f"Loss: {loss}, Accuracy: {accuracy}")

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7914 - loss: 0.5092 
Loss: 0.5468798279762268, Accuracy: 0.7520833611488342


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [24]:
# Set the model's file path
model_path = Path("student_loans.keras")

# Export your model to a keras file
model.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [29]:
# Set the model's file path
model_path = Path("student_loans.keras")

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [30]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[:5])

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[0.56796813]
 [0.31432566]
 [0.6472978 ]
 [0.63938695]
 [0.71272206]]


In [32]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=["predictions"])
predictions_df["predictions"] = predictions_df["predictions"].round()
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
475,1.0
476,1.0
477,0.0
478,1.0


### Step 4: Display a classification report with the y test data and predictions

In [33]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"]))

              precision    recall  f1-score   support

           0       0.76      0.71      0.73       230
           1       0.75      0.79      0.77       250

    accuracy                           0.75       480
   macro avg       0.75      0.75      0.75       480
weighted avg       0.75      0.75      0.75       480



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
Ans -To build a recommendation system for student loan options, you would need to collect the following data:

Student Demographics: Age, gender, location, socioeconomic background. This information helps tailor loan options to students' specific needs and financial situations.
Academic Performance: GPA, test scores, major, and academic progress. Students with strong academic records may qualify for better loan terms.
Financial Information: Income level, existing debt, credit score, financial aid status. This is crucial for assessing students' repayment ability and risk profile.
Loan Preferences: Preferred loan term (short-term vs. long-term), interest rate preferences (fixed vs. variable), and repayment plans. This helps in recommending loans that align with students' preferences and financial planning.
Loan History: Previous loans taken, repayment history, and any defaults. Historical data can provide insights into the student's financial responsibility and risk assessment.
Employment Status: Current job status, income, and job stability. Employed students or those with job offers may get better loan terms due to perceived lower risk.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
The model would be using content-based filtering. This method is suitable because the recommendation is based on the individual attributes and characteristics of each student, such as their demographics, academic performance, financial information, and loan preferences. Content-based filtering uses these features to find suitable loan options tailored to each student’s profile, without relying on the preferences and behaviors of other users.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Data Privacy and Security:

Concern: The recommendation system will handle sensitive personal and financial data. Ensuring data privacy and security is crucial to protect students' information from breaches and misuse.
Explanation: Breaches of sensitive data can lead to identity theft, financial fraud, and loss of trust in the system. Implementing robust security measures and compliance with data protection regulations (e.g., GDPR, CCPA) is essential.

Bias and Fairness:

Concern: The data used to train the recommendation system may contain biases that could lead to unfair loan recommendations. For example, students from certain demographics or socioeconomic backgrounds might be unfairly penalized.
Explanation: Bias in the system can lead to discriminatory practices, where some students are offered less favorable loan terms based on biased data. Ensuring fairness involves using unbiased data, regularly auditing the system for fairness, and implementing algorithms that mitigate bias.
By addressing these challenges, the recommendation system can provide equitable, secure, and reliable loan recommendations to all students.